# Real-time Steps

In real-time mode, the FPGA lets Xylo run freely and only performs readout after receiving a timestep done interrupt.
This means sending input to Xylo from the FPGA is not supported in real-time mode.  Instead, the switches on the PCB must be configured such that Xylo receives input from the microphones on the PCB.


Open Dev Kit

* Call open_device() on a discovered device with Samna.

In [ ]:
samna.device.open_device(d)

In [1]:
import warnings
warnings.filterwarnings("ignore")

import rockpool
print(f'Rockpool version {rockpool.__version__}')
import samna
print(f'Samna version {samna.__version__}')
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper, XyloMonitor, XyloSamna

from rockpool.devices.xylo import find_xylo_hdks

from rockpool.transform.quantize_methods import channel_quantize
from rockpool.nn.modules import LIF, Linear
from rockpool.nn.combinators import Sequential

import numpy as np

# find_xylo_hdks() directly calls the Samna device finder and opens all available devices.
# When Samna opens a device, it will perform actions on the board. Such as, powering on the chip, resetting it,
# applying a default configuration, and configuring the FPGA for default operation.

xylo_hdks, xylo_support_modules, xylo_versions = find_xylo_hdks()
print(xylo_hdks, xylo_support_modules, xylo_versions)

Rockpool version 2.7.dev
Samna version 0.37.10.26+g11d2fc3
from samna.xyloA3.configuration import InputInterfaceConfig
The connected Xylo HDK contains a Xylo A3. Importing `rockpool.devices.xylo.syns65302`
[<samna.xyloAudio3Boards.XyloAudio3TestBoard object at 0x7fe5abfd3cb0>] [<module 'rockpool.devices.xylo.syns65302' from '/home/vanessa/rockpool/rockpool/devices/xylo/syns65302/__init__.py'>] ['syns65302']


## Build a model and map it to hardware

In [2]:
# simple model to test

net = Sequential(
    Linear((16, 63)),
    LIF((63, 63)),

    Linear((63, 32)),
    LIF(32),
)

print(net)

# net[0].weight *= 0.05
spec = mapper(net.as_graph())
Q_spec = spec
Q_spec.update(channel_quantize(**Q_spec))
config, is_valid, msg = config_from_specification(**Q_spec)

if not is_valid:
    print(msg)

ModSequential  with shape (16, 32) {
    Linear '0_Linear' with shape (16, 63)
    LIF '1_LIF' with shape (63, 63)
    Linear '2_Linear' with shape (63, 32)
    LIF '3_LIF' with shape (32, 32)
}


## Set up Samna graphs and configuration

* Create a buffer sink for events from the chip model source node. Readout events will be captured here.
* Set Xylo configuration as desired for PDM and network config
* Set Xylo TR_WRAP register for desired time step length with default 25 MHz clock
* Set Xylo to real-time mode
* Enable PCB switches for PDM microphones
* Set FPGA module to real-time mode
   * io.write_config(0x12, 0) and io.write_config(0x31, 2)

In [3]:
# create xylo monitor
dt = 0.01
xylo_node = xylo_hdks[0]
print(xylo_node)


# PDM
config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm

# TR_WRAP register
config.time_resolution_wrap = int(0x7_9FF3)

# real-time mode
config.operation_mode = samna.xyloAudio3.OperationMode.RealTime

config.debug.debug_status_update_enable = 1
config.debug.debug_input_enable = 1
config.debug.monitor_signals_enable = 1

# PCB switches
# This is physically checked on the board

# inside XyloMonitor the buffers are created and FPGA modules are set
xylo_monitor = XyloMonitor(device=xylo_node, 
    config=config, 
    dt = dt,
    output_mode='Spike')

# print(xylo_monitor)

print(xylo_monitor.config)

# Create a XyloSamna structure for evolving
# modSamna = XyloSamna(xylo_node, config, dt = dt)

xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(0)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))


xyloAudio3::configuration::XyloConfiguration(operation_mode=OperationMode.Manual, bias_enable=1, time_resolution_wrap=499699, input=xyloAudio3::configuration::InputConfig(weight_bit_shift=0, weights={ 127 93 -3 112 15 50 91 -8 63 124 -18 116 -106 72 -93 57 45 -80 -111 -66 -68 -71 -27 -39 22 -83 22 76 120 122 91 117 -84 -114 -55 79 -60 -127 79 -72 54 -47 112 -17 15 -42 -16 30 -6 80 -71 117 57 -116 -126 -73 14 -58 93 46 29 -120 37 -31 20 -78 43 21 -112 20 88 -118 10 18 -50 9 78 95 124 127 -74 102 82 -89 -28 -81 -61 79 33 111 -27 -37 -123 40 -62 -63 12 56 45 -42 91 -64 9 -7 -27 114 -98 122 23 9 -105 86 -29 28 118 -73 111 -103 -34 99 56 -38 122 -68 25 60 -125 -14 94 35 84 -23 63 127 -99 -47 125 127 -127 52 48 93 -106 -58 88 92 114 -4 11 -44 38 99 -70 27 -18 38 127 47 110 13 -64 80 -72 -89 -91 -76 8 40 -104 -102 -84 -77 -102 -103 24 -127 80 32 -65 -56 -54 -31 -88 106 -64 -60 -20 -125 60 -8 127 -20 41 105 48 -127 33 -35 -35 -58 -44 0 127 96 99 -52 -102 -55 -45 -57 -82 94 -127 -19 -127 -79 -7

## Start processing and collect events

* Send TriggerProcessing event from Samna with the desired time step
* Collect readout events in the BufferSink attached to the model source node

In [4]:
# # The notebook calls XyloMonitor.evolve() with some input data, which the evolve function will try to send. 
# # This is not allowed in real-time mode.

# import numpy as np
# N = 1000
# a = 2
# b = float

# # in rd there is a AFE related record ('spike_in' or similar).
# # It should be empty in silence and non empty when we make noise
# out, _, rd = modSamna.evolve(input = np.zeros((N,2)), record = True)


# # out, _, rec = xylo_monitor.evolve(input_data=np.ones((N,2)))
# print(out)
# print(rd)


In [5]:
hex(1303)


'0x517'

In [9]:
xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(0)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[]


In [10]:
xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(12)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[]


In [11]:
xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(27)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[]


In [13]:
xylo_monitor._io.write_config(0x0008, 1)
